Web search results:

[1] "Define some parameters for the loader: batch_size = 32. img_height = 180. img_width = 180. It's good practice to use a validation split when developing your model. Use 80% of the images for training and 20% for validation. train_ds = tf.keras.utils.image_dataset_from_directory(."
URL: https://www.tensorflow.org/tutorials/images/classification

[2] "Lets assume the following source and destination paths: dir_src = r"/path/to/folder_with_subfolders" dir_dst = r"/path/to/destination_folder" The directory "folder_with_subfolders" contains multiple sub-folders and their associated files as shown in the tree diagram below:"
URL: https://stackoverflow.com/questions/68881569/how-to-copy-images-from-multiple-sub-folders-to-a-common-folder-and-rename-them

[3] "Its a stacked value defined above as -. images = np.vstack (images) This same prediction is being appended into images_data. Assuming your prediction is not failing, it means every prediction is the prediction on all the images stacked in the images_data. So, for every iteration for i in range (len (images_data)): This images_data [i] [0] is ..."
URL: https://datascience.stackexchange.com/questions/81541/how-to-predict-multiple-images-from-folder-in-python


Current date: 4/4/2023

Instructions: Using the provided web search results, write a comprehensive reply to the given query. Make sure to cite results using [[number](URL)] notation after the reference. If the provided search results refer to multiple subjects with the same name, write separate answers for each subject.
Query: I have a bunch of photos under different directories. The directories are classifications and the photos are training material. Create a python script that uses tensorflow keras to create a multiclass classifier that has layers for class map activation to have explainability into it's classification. Make sure to split the dataset into training and test and give the result of the model's accuracy at the end.

Based on the information provided in the web search results, I will help you create a Python script using TensorFlow Keras for creating a multiclass classifier. This classifier will use class map activation for explainability, and the dataset will be split into training and testing datasets. The model's accuracy will be displayed at the end.

First, let's import the necessary libraries and define the required parameters for the image loader:

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

batch_size = 32 #Batch size is a hyperparameter that defines the number of samples to work through before updating the internal model parameters.
img_height = 180
img_width = 180


To create a data preprocessing function that checks if the images are non-empty and have the correct dimensions, you can use the PIL library. If the images don't have the correct dimensions, this function will resize them to the specified height and width.

First, install the Pillow library if you haven't already:

In [6]:
!pip install Pillow

Now, let's create a data preprocessing function:


In [7]:
from PIL import Image

def preprocess_images(src_dir, dest_dir, img_height, img_width):
    os.makedirs(dest_dir, exist_ok=True)

    for subdir, dirs, files in os.walk(src_dir):
        for file in files:
            img_path = os.path.join(subdir, file)
            try:
                # Open the image
                img = Image.open(img_path)

                # Check if the image is empty
                if img.size == (0, 0):
                    print(f"Empty image {img_path}, skipping.")
                    continue

                # Resize the image if necessary
                if img.size != (img_width, img_height):
                    img = img.resize((img_width, img_height))

                # Save the preprocessed image to the destination directory
                dest_img_path = os.path.join(dest_dir, os.path.relpath(img_path, src_dir))
                os.makedirs(os.path.dirname(dest_img_path), exist_ok=True)
                img.save(dest_img_path)

            except Exception as e:
                print(f"Error processing image {img_path}: {e}")


To use the preprocess_images function, provide the source directory with the original images, the destination directory where the preprocessed images will be saved, and the desired image height and width:

In [8]:
src_dir = "D:\\devwork\\10dma\\extract.tar\\extract"
dest_dir = "D:\\devwork\\10dma\\extract.tar\\extract2"
img_height = 180
img_width = 180

import os
#If the output directory does not exist, create it
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

preprocess_images(src_dir, dest_dir, img_height, img_width)


KeyboardInterrupt: 

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image parameters
img_height = 180
img_width = 180
batch_size = 32

# Create an ImageDataGenerator instance for preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the pixel values to the [0, 1] range
    # Add other data augmentation options if desired, such as rotation, zoom, etc.
)

# Create a data generator for the training dataset
data_gen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
train_data = data_gen.flow_from_directory(
    'D:\\devwork\\10dma\\extract.tar\\extract',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
val_data = data_gen.flow_from_directory(
    'D:\\devwork\\10dma\\extract.tar\\extract',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 69958 images belonging to 307 classes.
Found 17350 images belonging to 307 classes.


Next, let's load the dataset and split it into training and validation sets. We'll use 80% of the images for training and 20% for validation as suggested in [1]:

In [2]:
data_dir = dest_dir
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)


Found 87308 files belonging to 307 classes.
Using 69847 files for training.
Found 87308 files belonging to 307 classes.
Using 17461 files for validation.


Now let's create a multiclass classifier model with class map activation for explainability:

In [10]:
num_classes = len(train_ds.class_names)

model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


Compile the model and define the loss function, optimizer, and evaluation metric:

In [11]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


Train the model using the training dataset and validate it with the validation dataset:

In [12]:
epochs = 10 #Epochs is the number of times the model will be trained on the entire dataset
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


Epoch 1/10


C:\Users\Nilekhet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


 375/2183 [====>.........................] - ETA: 12:52 - loss: 2.5599 - accuracy: 0.4552

InvalidArgumentError: Graph execution error:

Input is empty.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_3277]

Finally, let's display the model's accuracy:

In [ ]:
_, accuracy = model.evaluate(val_ds)
print(f"Model accuracy: {accuracy*100:.2f}%")


To make predictions on an image with explanations, you can use the Grad-CAM (Gradient-weighted Class Activation Mapping) technique. Grad-CAM generates a heatmap highlighting the regions in the input image that contributed to the prediction. First, install the opencv-python library if you haven't already:

In [ ]:
!pip install opencv-python

Now, let's create a function to make predictions with explanations:

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.models import Model

def load_image(path, target_size=(img_height, img_width)):
    img = image_utils.load_img(path, target_size=target_size)
    x = image_utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def grad_cam(model, img_array, layer_name):
    # Get the output of the desired layer
    output_layer = model.get_layer(layer_name).output

    # Create a model with the same input as the original model, but with the output of the target layer
    model_with_layer_output = Model(inputs=model.inputs, outputs=output_layer)

    # Calculate the gradients of the target layer with respect to the model's output
    with tf.GradientTape() as tape:
        inputs = tf.cast(img_array, tf.float32)
        tape.watch(inputs)
        outputs = model_with_layer_output(inputs)
        grads = tape.gradient(outputs, inputs)
        
    # Compute the guided gradients
    guided_grads = tf.cast(outputs > 0, "float32") * tf.cast(grads > 0, "float32") * grads

    # Calculate the weights for the guided gradients
    weights = tf.reduce_mean(guided_grads, axis=(1, 2))

    # Compute the Grad-CAM heatmap
    cam = np.ones(outputs.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights[0]):
        cam += w * outputs[0, :, :, i]
    cam = cv2.resize(cam.numpy(), (img_width, img_height))
    cam = np.maximum(cam, 0)
    heatmap = (cam - cam.min()) / (cam.max() - cam.min())

    # Return the heatmap
    return heatmap

def predict_with_explanation(model, image_path, layer_name="conv2d_2"):
    # Load and preprocess the image
    img_array = load_image(image_path, target_size=(img_height, img_width))

    # Make the prediction
    preds = model.predict(img_array)
    pred_class = np.argmax(preds[0])

    # Generate Grad-CAM heatmap
    heatmap = grad_cam(model, img_array, layer_name)

    # Load the original image
    original_img = cv2.imread(image_path)
    original_img = cv2.resize(original_img, (img_width, img_height))

    # Superimpose the heatmap on the original image
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + original_img

    # Display the original image, heatmap, and superimposed image
    cv2.imshow("Original Image", original_img)
    cv2.imshow("Heatmap", heatmap)
    cv2.imshow("Superimposed Image", superimposed_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Return the prediction class and its probability
    return train_ds.class_names[pred_class], preds[0][pred_class]


Now you can use the predict_with_explanation function to make predictions on an image with explanations. The function takes the trained model, the image path, and the name of the layer you want to use for generating the heatmap (by default, it uses the last convolutional layer in the example model).

Here's an example of how to use the function with a test image:

In [ ]:
image_path = "path/to/your/test_image.jpg"
predicted_class, probability = predict_with_explanation(model, image_path)
print(f"Predicted class: {predicted_class}, Probability: {probability:.2f}")


The predict_with_explanation function will display the original image, the heatmap, and the superimposed image with the heatmap. It will also return the predicted class and its probability.

You can save the model using the save method and load it back using the load_model function from tensorflow.keras.models. Here's a function to save the model and another function to make predictions with explanations using the saved model:

In [ ]:
from tensorflow.keras.models import load_model

def save_trained_model(model, model_path):
    model.save(model_path)

def load_saved_model(model_path):
    return load_model(model_path)

def predict_with_saved_model(model_path, image_path, layer_name="conv2d_2"):
    # Load the saved model
    saved_model = load_saved_model(model_path)

    # Make a prediction with explanation using the saved model
    predicted_class, probability = predict_with_explanation(saved_model, image_path, layer_name)

    return predicted_class, probability


To save your trained model, use the save_trained_model function:

In [ ]:
model_path = "path/to/your/saved_model"
save_trained_model(model, model_path)


Now, you can move the saved model to a different computer. To make predictions with explanations using the saved model, use the predict_with_saved_model function:

In [ ]:
image_path = "path/to/your/test_image.jpg"
predicted_class, probability = predict_with_saved_model(model_path, image_path)
print(f"Predicted class: {predicted_class}, Probability: {probability:.2f}")


Replace model_path with the path to the saved model and image_path with the path to the image you want to make a prediction on. The predict_with_saved_model function will use the saved model to make predictions with explanations as described in the previous response.